In [1]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv("Train_UWu5bXk.csv")
test = pd.read_csv("Test_u94Q5KV.csv")

In [ ]:
train.Item_Outlet_Sales.describe()

In [ ]:
train.isna().sum()

In [ ]:
train.columns

In [ ]:
train.Outlet_Type.unique()

In [3]:
train.Item_Fat_Content.replace(['low fat','LF','reg'],['Low Fat','Low Fat','Regular'], inplace=True)
test['Item_Fat_Content'].replace(['low fat','LF','reg'],['Low Fat','Low Fat','Regular'],inplace = True)

In [4]:
train["Item_Weight"] = train["Item_Weight"].fillna(train['Item_Weight'].dropna().mode().values[0])
train["Outlet_Size"] = train["Outlet_Size"].fillna(train['Outlet_Size'].dropna().mode().values[0])
test["Item_Weight"] = test["Item_Weight"].fillna(test['Item_Weight'].dropna().mode().values[0])
test["Outlet_Size"] = test["Outlet_Size"].fillna(test['Outlet_Size'].dropna().mode().values[0])

In [ ]:
sns.pairplot(train)

In [ ]:
sns.heatmap(train.corr(),annot=True)

In [5]:
X1 = train.drop(["Item_Identifier","Outlet_Identifier"],axis=1)
X2 = test.drop(["Item_Identifier","Outlet_Identifier"],axis=1)

X = X1.iloc[:,:-1]
y = X1.iloc[:,-1]
X_cv = X2

# Dummy variables
X = pd.get_dummies(X, drop_first=True)
X_cv = pd.get_dummies(X_cv, drop_first=True)

In [6]:
# Spliting data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [8]:
# feature scaling
from  sklearn.preprocessing  import StandardScaler

slc= StandardScaler()
X_train = slc.fit_transform(X_train)
X_cv = slc.transform(X_cv)
X_test = slc.transform(X_test)

r2 = {}
rmse ={}

In [9]:
# SVR model
from sklearn.svm import SVR
svr = SVR()
svr.fit(X_train, y_train)

y_predict = svr.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score
rmse['svr'] = np.sqrt(mean_squared_error(y_test, y_predict))
r2['svr'] = r2_score(y_test,y_predict)

In [10]:
# creating XGBoost model
from xgboost.sklearn import XGBRegressor
XGB = XGBRegressor()
XGB.fit(X_train, y_train)
y_predict = XGB.predict(X_test)

rmse['xgb'] = np.sqrt(mean_squared_error(y_test, y_predict))
r2['xgb'] = r2_score(y_test,y_predict)

In [11]:
# linear regression model 
from sklearn.linear_model import LinearRegression 
lr = LinearRegression() 
lr.fit(X_train, y_train) 
y_predict = lr.predict(X_test)

rmse['lr'] = np.sqrt(mean_squared_error(y_test, y_predict))
r2['lr'] = r2_score(y_test,y_predict)

In [12]:
# Decision Tree regression model
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train) 
y_predict = dtr.predict(X_test)

rmse['dtr'] = np.sqrt(mean_squared_error(y_test, y_predict))
r2['dtr'] = r2_score(y_test,y_predict)

In [13]:
# Random Forest regression model
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train) 
y_predict = rfr.predict(X_test)

rmse['rfr'] = np.sqrt(mean_squared_error(y_test, y_predict))
r2['rfr'] = r2_score(y_test,y_predict)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [14]:
svr.score(X_train,y_train), XGB.score(X_train,y_train), lr.score(X_train,y_train), dtr.score(X_train,y_train), rfr.score(X_train,y_train)

(-0.00564945604849898,
 0.6337853614292461,
 0.5634853028191205,
 1.0,
 0.9156342067780219)

In [15]:
r2

{'svr': -0.010950363895292803,
 'xgb': 0.587902272186229,
 'lr': 0.562065073173319,
 'dtr': 0.21305106072962487,
 'rfr': 0.5243817278634667}

In [16]:
rmse

{'svr': 1720.1481415630726,
 'xgb': 1098.2495755279024,
 'lr': 1132.154574390424,
 'dtr': 1517.6590334488699,
 'rfr': 1179.8592440406114}

In [17]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(XGB, X_train, y_train, cv=10)
scores

array([0.59986743, 0.60010084, 0.56203252, 0.56425125, 0.61334511,
       0.60859078, 0.59688925, 0.60732306, 0.58644091, 0.58998615])

In [18]:
scores.mean(), scores.std()

(0.5928827294850836, 0.016769910394309227)

In [25]:
from sklearn.grid_search import GridSearchCV
params = {}
grid_search = GridSearchCV(estimator = XGB, param_grid = params,n_jobs = -1,scoring = "accuracy", cv=10) 
gs = grid_search.fit(X_train, y_train)

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\ProgramData\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\ProgramData\Anaconda3\lib\runpy.py

In [ ]:
rf_best_accuracy = grid_search.best_score_
rf_best_parameters = grid_search.best_params_
rf_best_accuracy, rf_best_parameters

In [ ]:
y_results = XGB.predict(X_cv)

In [ ]:
final = pd.read_csv("SampleSubmission_TmnO39y.csv")

In [ ]:
final.head()

In [ ]:
final.Item_Outlet_Sales = y_results

In [ ]:
final.to_csv("my_submission1.csv")

## XGBoost gives the least RSME value